## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys

import envbash
envbash.load.load_envbash('../env')

sys.path.append('/Users/yvesmartin/src/rouskinhf')

from rouskinhf import convert, upload_dataset

# Create a datafolder from local files

These methods will allow you to process your data and create a datafolder from it. The accepted formats are:
- DREEM output
- fasta
- set of CTs
- already formatted json + info.json

**Make sure to change the paths to your own paths!**

### From SEISMIC-json format

In [34]:
%reload_ext autoreload
%autoreload 2
# Show all arguments
data = convert(format='seismic', 
              file_or_folder='../data/input_files_for_testing/test_dreem_output.json',
              name='seismic_output',
              path_out='../data/testing',
              predict_structure=True,
              filter=True,
              min_AUROC=0.5,   
              verbose=True)

Parsing dreem output file: 100%|██████████| 13/13 [00:01<00:00,  8.55it/s]

Over a total of 13 datapoints, there are:
    OUTPUT
    - 10 valid datapoints
    MODIFIED
    - 0 multiple sequences with the same reference (renamed reference)
    FILTERED OUT
    - 1 invalid datapoints (ex: sequence with non-regular characters)
    - 0 datapoints with bad structures
    - 0 duplicate sequences with the same structure / dms
    - 0 duplicate sequences with different structure / dms
    - 2 datapoints removed because of low AUROC (<0.5)


### From a data.json file

In [30]:
data = convert(format='json', 
              file_or_folder='../data/input_files_for_testing/data.json',
              path_out='data/testing',
              predict_structure=True,
              filter=True,   
              min_AUROC=0.5,
              verbose=True)

Parsing json file: 100%|██████████| 10/10 [00:00<00:00, 17667.67it/s]

Over a total of 10 datapoints, there are:
    OUTPUT
    - 10 valid datapoints
    MODIFIED
    - 0 multiple sequences with the same reference (renamed reference)
    FILTERED OUT
    - 0 invalid datapoints (ex: sequence with non-regular characters)
    - 0 datapoints with bad structures
    - 0 duplicate sequences with the same structure / dms
    - 0 duplicate sequences with different structure / dms
    - 0 datapoints removed because of low AUROC (<0.5)


### From a list of CT files

In [32]:
%reload_ext autoreload
%autoreload 2
data = convert(format='ct', 
              file_or_folder='../data/input_files_for_testing/test_ct_files',
              path_out='data/testing',
              predict_structure=True)

Parsing ct files: 100%|██████████| 5/5 [00:00<00:00, 6009.03it/s]

Over a total of 5 datapoints, there are:
    OUTPUT
    - 2 valid datapoints
    MODIFIED
    - 0 multiple sequences with the same reference (renamed reference)
    FILTERED OUT
    - 1 invalid datapoints (ex: sequence with non-regular characters)
    - 0 datapoints with bad structures
    - 1 duplicate sequences



### From fasta

In [36]:
data = convert(format='fasta', 
              file_or_folder='../data/input_files_for_testing/test_sequences.fasta',
              path_out='../data/testing',
              predict_structure=True,
              filter=True,   
              verbose=True)

Parsing fasta file: 100%|██████████| 5/5 [00:00<00:00, 26.20it/s]

Over a total of 5 datapoints, there are:
    OUTPUT
    - 3 valid datapoints
    MODIFIED
    - 1 multiple sequences with the same reference (renamed reference)
    FILTERED OUT
    - 1 invalid datapoints (ex: sequence with non-regular characters)
    - 0 datapoints with bad structures
    - 0 duplicate sequences


# Push a local datafolder to Hugging Face

In [3]:

# Find more arguments here: https://huggingface.co/docs/huggingface_hub/guides/upload#upload-a-folder
upload_dataset(
    datapath='../data/testing/test_sequences/data.json', 
    exist_ok=True,
    commit_message='test upload',
)

### 3. Check that the datafolder is on Hugging Face

Take a look at https://huggingface.co/rouskinlab

# Explore the datafolder object